# Analisando acidentes fatais na estrada na Austrália entre 1989 e 2021


In [77]:
#TODO DESCOMENTAR PARA ENTREGA
# pd.set_option('display.max_columns', None)
# path_to_arthurs_drive = '/content/drive/MyDrive/IA - P3/Crash_Data.csv'
# df = pd.read_csv(path_to_arthurs_drive)

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder
# from sklearn.impute import KNNImputer
# from matplotlib.dates import DateFormatter

import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv("Crash_Data.csv")


# Visualizando os dados brutos

In [78]:
df.head(5)

In [79]:
# Emite um relatório simples sobre TODAS AS COLUNAS do Data frame
for column in df:
  soma_valores_nulos = df[column].isnull().sum()
  total_valores = len(df[column])
  porcentagem_nulos = round((soma_valores_nulos / total_valores) * 100, 2)

  print(f'Coluna {column} possui {soma_valores_nulos} valores nulos do total de {total_valores} -> {porcentagem_nulos}% \n valores diferentes: {set(df[column])} \n\n')


# Tratando colunas selecionadas para análise

In [80]:
# Separa as colunas que serão usadas para análise e descarta as demais
df = df[[
  'Time',
  'Time of day',
  'Day of week',
  'Month',
  'Year',
  'Dayweek',
  'Crash Type',
  'Road User',
  'Gender',
  'Age',
  'Age Group',
  'State',
  'Speed Limit',
  'Bus Involvement',
  'Articulated Truck Involvement',
  'Christmas Period',
  'Easter Period',
]]

# Emite um relatório simples sobre os dados de cada coluna do Data frame
for column in df:
  soma_valores_nulos = df[column].isnull().sum()
  total_valores = len(df[column])
  porcentagem_nulos = round((soma_valores_nulos / total_valores) * 100, 2)

  print(f'Coluna {column} possui {soma_valores_nulos} valores nulos do total de {total_valores} -> {porcentagem_nulos}% \n valores diferentes: {set(df[column])} \n\n')



In [74]:
dfcopy = df.copy()
# Drop nas linhas onde a existe ao menos um valor nulo
dfcopy.dropna()

In [ ]:
dfcopy.isnull().sum()